<a href="https://colab.research.google.com/github/w4bo/handsOnDataPipelines/blob/main/materials/01-DataPreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Integrated analytics lab



Data analysis involves several steps:

1. Frame the problem and look at the big picture
   - Define the objective in business terms
   - How should performance be measured?
1. Get the data
   - List the data you need and how much you need
   - Collect of data from one or more sources (database, web, etc.)
     - In this lab, things are much simpler
1. Explore the data to gain insights
   - Create an environment to keep track of your data exploration
     - You have been provided with notebook environments
   - Understanding of the structure and meaning of data
1. Transformation of data into manageable formats for subsequent steps
1. Extraction of knowledge from data (statistics, models, patterns, etc.)
1. Validation of the extracted knowledge
1. Deployment of the extracted knowledge and models


![image](https://upload.wikimedia.org/wikipedia/commons/thumb/b/b9/CRISP-DM_Process_Diagram.png/1024px-CRISP-DM_Process_Diagram.png)

# Relational data

**Relational data** are usually collected in **tabular** format

- Each row is an **observation** (instance or tuple)
    - An object of the analysis
    - E.g., a product for market basket analysis
- Each column is an **attribute** (or feature) characterizing each object
    - All values within a column have the same type (i.e., all values belong to the same attribute domain)
    - E.g., the attributes ID (int), ProductName (str), or Price (float)

**Pandas** is the shorthand for 'Python and Data Analysis'

- It provides a rich set of features for exploring and manipulating data
- https://pandas.pydata.org/

pandas (Python) is a solution for the manipulation of relational data

- Two main data types: Series (e.g., temporal series) and DataFrame (e.g., table)
- Support to SQL-like operations (join/merge, aggregation, etc.)
- Imputation of missing values
- Manipulation of data shape
- By convention, the package pandas is imported as “pd”


In [1]:
import pandas as pd
print(pd.__version__)

1.5.3


... plus we will use other libraries

In [2]:
import numpy as np  # fast operations on arrays
import seaborn as sns  # plots
import matplotlib.pyplot as plt  # plots

Pandas relies on DataFrame and Series



**DataFrame**:
- Two-dimensional, size-mutable, potentially heterogeneous tabular data.
- The primary pandas data structure.
- Data structure also contains labeled axes (rows and columns).
- Arithmetic operations align on both row and column labels.
- Can be thought of as a dict-like container for Series objects.
- https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html

In [3]:
# create a numeric dataframe/table
df = pd.DataFrame([[i + j for i in range(10)] for j in range(5)],
                  index=[i for i in range(5)],
                  columns=list('abcdefghij'))
df

,a,b,c,d,e,f,g,h,i,j
0,0,1,2,3,4,5,6,7,8,9
1,1,2,3,4,5,6,7,8,9,10
2,2,3,4,5,6,7,8,9,10,11
3,3,4,5,6,7,8,9,10,11,12
4,4,5,6,7,8,9,10,11,12,13


A **Series** is a sequence of values with the same type

- Each value is associate with a label
- Supported values and label types are the ones from NumPy (float64, int64, etc.)
- In other words, a series is a mono-dimensional vector of elements

The index of a series is the sequence of labels

- Label are usually numeric or string identifiers
- E.g., the primary key of a database table
- Labels could repeat within the series, but usually do not

Technically

- One-dimensional ndarray with axis labels (including time series).
- Labels need not be unique but must be a hashable type (both integer- and label-based indexing).
- Operations between Series (e.g., +, -, /) align values based on their associated index values.
- https://pandas.pydata.org/docs/reference/api/pandas.Series.html

In [4]:
pd.Series([1, 2, 3], index=["a", "b", "c"])

a    1
b    2
c    3
dtype: int64

In [5]:
# Select a column (i.e., a series) as in a SQL projection
df['a']

0    0
1    1
2    2
3    3
4    4
Name: a, dtype: int64

In [6]:
# Add another column to the dataframe
df['k'] = df['a'] * df['b']
df

,a,b,c,d,e,f,g,h,i,j,k
0,0,1,2,3,4,5,6,7,8,9,0
1,1,2,3,4,5,6,7,8,9,10,2
2,2,3,4,5,6,7,8,9,10,11,6
3,3,4,5,6,7,8,9,10,11,12,12
4,4,5,6,7,8,9,10,11,12,13,20


... doing some element-wise operations

In [ ]:
ser_a = pd.Series([1, 2, 3], index=["a", "b", "c"])
ser_b = pd.Series([1, 2, 3], index=["b", "a", "c"])
ser_a + ser_b

In [ ]:
ser_a - ser_b

In [ ]:
ser_a * ser_b

In [ ]:
ser_a / ser_b

... doing some aggregation 

In [ ]:
ser_c = df["a"]
ser_c.count()  # => 5
ser_c.sum()    # => 10
ser_c.mean()   # => 2.0
ser_c.max()    # => 4
ser_c.min()    # => 0
ser_c.idxmax() # => 4

## Data preprocessing

Data preprocessing plays a key role in a data analytics process and avoids “Garbage in, garbage out” [1]

- A broad range of activities; from correcting errors to selecting the most relevant features
- There are no pre-defined rules on the impact of pre-processing transformations
- Data scientists cannot easily foresee the impact of pipeline prototypes

“Garbage in, garbage out” is particularly applicable to data mining and machine learning
- Out-of-range values (e.g., Income: −100)
- Impossible data combinations (e.g., Exam mark: 15, Exam result: Passed) 
- Missing values
- Inconsistent data among multiple sources
- More?
    
[1] Joseph Giovanelli, Besim Bilalli, Alberto Abelló: Effective data pre-processing for AutoML. DOLAP 2021: 1-10

Which transformations can we apply?
- **Encoding**: transforming categorical attributes into continuous ones
- **Discretization**: transforming continuous attributes into categorical ones
- **Normalization**: normalizing continuous attributes such that their values fall in the same range
- **Imputation**: imputing missing values
- **Rebalancing**: adjusting the class distribution of a dataset (i.e., the ratio between the different classes/categories represented)
- **Feature Engineering**: defining the set of relevant attributes (variables, predictors) to be used in model construction

## Understanding data types

"It is imperative to know the attribute properties to carry out meaningful operations and research with them"

Why is data type important?

![image](https://user-images.githubusercontent.com/18005592/232748093-a25e8ba7-24d4-4e2b-9e58-1553786cac33.png)


A signed integer is a 32-bit datum that encodes an integer in the range:

$[-2^{31}, 2^{31}-1] = [-2147483648, 2147483647]$

$2201010001 > 2147483647$

In [ ]:
df = pd.DataFrame([
    ['Cola',   'low',    '05/07/2021', 10],
    ['Bread',  'medium', '05/07/2021', 25],
    ['Beer',   'high',   '06/07/2021', 100],
    ['Diaper', 'high',   '06/07/2021', np.nan],
    ['Pizza',  'medium', '06/07/2021', 25]], columns=['ID', 'PriceBin', 'Date', 'Quantity'])
df

The attribute type determines which operator can be applied to the attribute
- Equality, sort, sum, ratio, etc.
- It makes sense to compute the average `Quantity` but not the average `ID`

Different attribute types

- (Categorical) **Nominal**: can distinguish the values (i.e., check equality)
- (Categorical) **Ordinal**: can distinguish and sort the values
- (Numeric) **Interval**: can distinguish and sort the values, and compute their difference
- (Numeric) **Ratio**: can distinguish and sort the values, and compute their difference and ratio

Pandas automatically infers data types, or they can be specified during creation
- Common data types are numeric ones
    - `np.floatN` represents floating numbers (e.g., -3.14)
    - `np.intN`/`np.uintN` represent integers with/without sign (-42 and 42)
    - `N` is the number of needed bits: 8, 16, 32 o 64
- Other data types
    - `bool`: Boolean values
    - `datetime64`, timedelta64: timestamp and time intervals
    - `object`: mainly used for strings

In [ ]:
df.dtypes

In [ ]:
# data profiling
df.info()

In [ ]:
# To get some statistics (e.g., count, mean, std, min, etc.)
df.describe(include='all')

In [ ]:
# Get the headers (i.e., the column names)
df.columns

# Get just the first two rows
df.head(2)

# Get just the last two rows
df.tail(2)

# Sort the dataframe by columns
df.sort_values(by=['Quantity', 'ID'], ascending=[False, True])

## Data distribution

In [ ]:
# array with distinct values sorted by first appearance
df["Quantity"].unique() 

In [ ]:
# quantity of unique values
df["Quantity"].nunique()

In [ ]:
# return a new series that associates each value with its number of occurrences, sorted by frequency
df["Quantity"].value_counts()

In [ ]:
df["Quantity"].hist(bins=10)

Which problems can cause skewed distributions?

![image](https://user-images.githubusercontent.com/18005592/232750742-aacbf6b3-8a7d-49c6-b253-5ab8e7985104.png)

Things are even more complex when applying sequences of transformations

- E.g., normalization should be applied before rebalancing since rebalancing (e.g., by resampling) alters average and standard deviations
- E.g., applying feature engineering before/after rebalancing produces different results which depends on the dataset and the algorithm

![image](https://user-images.githubusercontent.com/18005592/232754117-8a84fde5-bce2-41b1-a003-7dfa0b63f980.png)

More an art than a science
- ... At least for now


## Missing values

Datasets often show missing values
- E.g., they are not applicable (e.g., date of death) or unknown
- A series can have missing values, referred to as `NA` (Not Available)
- Numeric attributes: `NA` is `np.nan` (Not a Number)
- `nan` is never equal, greater, or lower than other values (nor itself)

        np.nan == np.nan
        False
- Numeric expressions with `nan` return `nan`

        2 * np.nan – 1
        nan
Which problems arise from missing values?

In [ ]:
# add the column "isna", True if the value is NaN
df["isna"] = df["Quantity"].isna()
# add the column "notna", False if the value is NaN
df["notna"] = df["Quantity"].notna()
df[["Quantity", "isna", "notna"]]

### Imputing missing values

Several strategies

- Replace `nan` with average or median values
- Forward/backward fill
- Dropping rows/columns with nans

Which are the effects?

In [ ]:
# fill the missing value with the average
df["Quantity_imputed"] = df["Quantity"].fillna(df["Quantity"].mean())  # fillna replaces NA values
df[["Quantity", "Quantity_imputed"]]

In [ ]:
# fill the missing value with the previous (not NaN) value
df["Quantity"].fillna(method="ffill")

In [ ]:
# fill the missing value with the following (not NaN) value
df["Quantity"].fillna(method="bfill")

In [ ]:
df["Quantity"].dropna()

In [ ]:
df.dropna()

## The `Housing` case study

Check also: 

- https://www.kaggle.com/camnugent/california-housing-prices
- https://www.oreilly.com/library/view/hands-on-machine-learning/9781492032632/

We will use the California Housing Prices dataset.
Our task is to use California census data to forecast housing prices given the population, median income, and median housing price for each block group in California.
Block groups are the smallest geographical unit for which the US Census Bureau publishes sample data (a block group typically has a population of 600 to 3,000 people).
We will just call them "districts" for short

In [ ]:
# df = pd.read_csv("datasets/2022-bbs-dsaa-housing.csv", delimiter=",")
df = pd.read_csv("https://raw.githubusercontent.com/w4bo/handsOnDataPipelines/main/materials/datasets/housing.csv", delimiter=",")
df

... and now?

Answer some questions:

- Which attributes (i.e., columns) are contained in the dataset?
- Which is their semantics?

In [ ]:
df.columns

Dataset description

1. `longitude`: A measure of how far west a house is; a higher value is farther west
2. `latitude`: A measure of how far north a house is; a higher value is farther north
3. `housingMedianAge`: Median age of a house within a block; a lower number is a newer building
4. `totalRooms`: Total number of rooms within a block
5. `totalBedrooms`: Total number of bedrooms within a block
6. `population`: Total number of people residing within a block
7. `households`: Total number of households, a group of people residing within a home unit, for a block
8. `medianIncome`: Median income for households within a block of houses (measured in tens of thousands of US Dollars)
9. `medianHouseValue`: Median house value for households within a block (measured in US Dollars)
10. `oceanProximity`: Location of the house w.r.t ocean/sea

In [ ]:
# show some statistics on the dataframe
df.info()

In [ ]:
df.describe(include='all')

... are you satisfied with the understanding?

... what about data visualization?

In [ ]:
df.plot(kind="scatter", x="longitude", y="latitude", alpha=0.4, s=df["population"]/100, label="population", figsize=(10,7), c="median_house_value", cmap="jet", colorbar=True)

What if we integrate open data?


![image](https://user-images.githubusercontent.com/18005592/232756567-b706619a-2cc9-4b45-b78f-5172103e0c3b.png)

#### Memory usage

What if I change float64 to float32?

In [ ]:
dff = df.copy(deep=True)  # copy the dataframe
for x in df.columns:  # iterate over the columns
    if dff[x].dtype == 'float64':  # if the column has type `float64`
        dff[x] = dff[x].astype('float32')  # ... change it to `float32`
dff.info()  # show some statistics on the dataframe

#### Missing values
There are some missing values for `total_bedrooms`. What should we do?

Most Machine Learning algorithms cannot work with missing features. We have three options:
- Get rid of the corresponding districts (i.e., drop the rows)
    - `df.dropna(subset=["total_bedrooms"])`
- Get rid of the whole attribute (i.e., drop the columns)
    - `df.drop("total_bedrooms", axis=1`
- Set the values to some value (zero, the mean, the median, etc.)
    - `df["total_bedrooms"].fillna(df["total_bedrooms"].median())`

#### Non-numeric attributes
`ocean_proximity` is a text attribute so we cannot compute its median. Some options:
- Get rid of the whole attribute. (`df.drop("ocean_proximity", axis=1`)
- Change from categorical to ordinal (e.g., `NEAR BAY` = 0, `INLAND` = 1)
    - Can foresee any problem in this?
    - ML algorithms will assume that two nearby values are more similar than two distant values. This may be fine in some cases (e.g., for ordered categories such as “bad”, “average”, “good”, “excellent”), but it is obviously not the case for the ocean_proximity column (for example, categories 0 and 4 are clearly more similar than categories 0 and 1). 
- Change from categorical to one hot encoding
    - To fix this issue, a common solution is to create one binary attribute per category: one attribute equal to 1 when the category is “<1H OCEAN” (and 0 otherwise), another attribute equal to 1 when the category is “INLAND” (and 0 otherwise), and so on. This is called one-hot encoding, because only one attribute will be equal to 1 (hot), while the others will be 0 (cold). The new attributes are sometimes called dummy attributes

In [ ]:
df["ocean_proximity"].value_counts()

In [ ]:
df["ocean_proximity"].hist()

Change from categorical to ordinal

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
y = ordinal_encoder.fit_transform(df[["ocean_proximity"]])
y

From categorical to one-hot encoding

In [ ]:
y = pd.get_dummies(df["ocean_proximity"], prefix='ocean_proximity')
y

### Visualization

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
df.hist(bins=50, figsize=(20, 15))
plt.show()

Open questions:

- `median_income` should be in dollars. However, it has a strange range. Why? "you are told that the data has been scaled and capped at 15 (actually 15.0001) for higher median incomes, and at 0.5 (actually 0.4999) for lower median incomes. The numbers represent roughly tens of thousands of dollars. The numbers represent roughly tens of thousands of dollars"
- `housing_median_age` and `median_house_value` are capped. As to `median_house_value`, this is a serious problem since it is your target attribute (your labels). Your Machine Learning algorithms may learn that prices never go beyond that limit. You need to check with your client team (the team that will use your system’s output) to see if this is a problem or not. If they tell you that they need precise predictions even beyond 500,000USD, then you have mainly two options: (a) collect proper labels for the districts whose labels were capped, (b) remove those districts from the training set."
- These attributes have very different scales. Should we scale them?
- Many histograms are tail heavy: they extend much farther to the right of the median than to the left. This may make it a bit harder for some Machine Learning algorithms to detect patterns

Are the relationships between variables?

- A grid of Axes such that each numeric variable in data will by shared across the y-axes across a single row and the x-axes across a single column
- The diagonal plots are treated differently: a univariate distribution plot is drawn to show the marginal distribution of the data in each column.


In [ ]:
tmp = df[["median_income", "housing_median_age", "median_house_value", "households", "population", "total_rooms"]]
sns.pairplot(tmp.sample(n=1000, random_state=42), hue='median_house_value', markers='+')
plt.show()

Check correlations and intervals

In [ ]:
from scipy.stats import pearsonr
rho = df.corr(method='pearson')
pval = df.corr(method=lambda x, y: pearsonr(x, y)[1]) - np.eye(*rho.shape)
p = pval.applymap(lambda x: ''.join(['*' for t in [0.01, 0.05, 0.1] if x <= t]))
rho.round(2).astype(str) + p

In [ ]:
min_corr = 0.3
kot = rho[(abs(rho) >= min_corr) & (rho < 1)]
plt.figure(figsize=(8, 6))
sns.heatmap(kot, cmap=sns.color_palette("coolwarm", as_cmap=True))

#### Scaling attributes

Attributes have very different scales. Should we scale them?


- Normalization is good to use when you know that the distribution of your data does not follow a Gaussian distribution. This can be useful in algorithms that do not assume any distribution of the data like K-Nearest Neighbors and Neural Networks.
- Standardization, on the other hand, can be helpful in cases where the data follows a Gaussian distribution. However, this does not have to be necessarily true. Unlike normalization, standardization does not have a bounding range. So, even if you have outliers in your data, they will not be affected by standardization.


#### Min-max normalization

In [ ]:
num_df = df.drop(columns=['ocean_proximity', 'median_house_value'])
normalized_df = (num_df - num_df.min()) / (num_df.max() - num_df.min())
normalized_df

#### Standardization

In [ ]:
num_df = df.drop(columns=['ocean_proximity', 'median_house_value'])
normalized_df = (num_df - num_df.mean()) / num_df.std()
normalized_df

This checklist can help you while building your projects
- Frame the problem and look at the big picture
   - ✔ Define the objective in business terms
   - ✖ How should performance be measured?
- Get the data
   - ✔ List the data you need and how much you need
- Explore the data to gain insights
   - ✔ Create an environment to keep track of your data exploration 
   - ✔ Study each attribute and its characteristics
- Prepare the data
   - ✔ Fix or remove outliers (optional)
   - ✔ Fill in missing values (e.g., with zero, mean, median…) or drop their rows (or columns)
   - ✔ Feature selection (optional): drop the attributes that provide no useful information for the task
   - ✔ Feature engineering, where appropriate: discretize continuous features


#### Hands on!

In [ ]:
num_df = df.copy(deep=True).drop(columns=["ocean_proximity"])  # do not change this line

# Filling in (i.e., impute) missing values with the median value
num_df["total_bedrooms"] = 1  # change `1` with the proper solution

# Add a new column: population_per_household = population / households
num_df["population_per_household"] = 1  # change `1` with the proper solution

# Add a new column: rooms_per_household = total_rooms / households
num_df["rooms_per_household"] = 1  # change `1` with the proper solution

# Add a new column: bedrooms_per_room = total_bedrooms / total_rooms
num_df["bedrooms_per_room"] = 1  # change `1` with the proper solution

# Apply standardization to all the numeric columns
num_df = pd.DataFrame()  # change `pd.DataFrame()` with the proper solution

# One hot encode `ocean_proximity` since it is a categorical attribute
# change `pd.DataFrame()` with the proper solution (hint: pd.get_dummies)
cat_df = pd.DataFrame()

clean_df = pd.concat([num_df, cat_df], axis=1)  # do not change this line
clean_df